# Final assignment - Python
## Linoy Medalsy - 206073124
## Amit Fallach - 318510070

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_val_score, KFold
from datetime import datetime, timedelta
import string
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import ppscore as pps
import statsmodels.api as sm
from sklearn.inspection import permutation_importance

C:\Users\amitf\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\amitf\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
#path_linoy = "C:/Users/linoy/OneDrive/שולחן העבודה/לימודים/תואר/שנה ג/סמסטר ב/כרייה וניתוח נתונים מתקדם בפייתון/מטלות/output_all_students_Train_v9.csv"
#data = pd.read_csv(path_linoy)


#path_amit = 'C:\\Users\\amitf\\Desktop\\amit\\למידת מכונה\\final_ass\\output_all_students_Train_v10.csv'
#data = pd.read_csv(path_amit)

excel_file = 'Dataset_for_test.xlsx'
data = pd.read_excel(excel_file)


data = data.copy()
data

,City,type,room_number,Area,Street,number_in_street,city_area,price,num_of_images,floor_out_of,...,hasStorage,condition,hasAirCondition,hasBalcony,hasMamad,handicapFriendly,entranceDate,furniture,publishedDays,description
0,פתח תקווה,בית פרטי,4,600,עזרא ונחמיה,44.0,עמישב,6999999,2,קומת קרקע,...,False,משופץ,True,False,False,False,לא צויין,מלא,NaN,"דירת מרווחת ומוארת במצב מצוין, 4 חדרים גדולים,..."
1,דימונה,דירה,2,45,דיין משה,117.0,נאות קטיף,520000,0,קומה 1 מתוך 4,...,False,לא צויין,True,False,False,False,לא צויין,מלא,11.0,אזור טוב ושקט בדימונה.שכנים נוחים.הדירה מושכרת...
2,ירושלים,דירה,3 חד׳,"94 מ""ר",רבדים,6.0,ארנונה (תלפיות),3450000,11,קומה 8 מתוך 8,...,כן,משופץ,כן,כן,לא,כן,לא צויין,חלקי,NaN,"הדירה משופצת ומושקעת , נמצאת בקרבת תחבורה ציב..."
3,הרצליה,דירה,5,135,פינס,5.0,הרצליה הצעירה,3800000,0,קומה 1 מתוך 3,...,יש מחסן,משופץ,יש מיזוג אויר,אין מרפסת,אין ממ״ד,לא נגיש לנכים,לא צויין,לא צויין,NaN,דירה גדולה מאוד יחידה בקומה. משופצת כמו חדשה. ...
4,רעננה,דירת גן,5 חד׳,"139 מ""ר",עלי מוהר,9.0,נווה זמר,5555555,4,קומת קרקע,...,יש מחסן,חדש מקבלן,יש מיזוג אויר,אין מרפסת,יש ממ״ד,לא נגיש לנכים,2025-02-01 00:00:00,ללא,NaN,דירת גן בפרוייקט יוקרתי בבניין בוטיק רק 12 דיר...
5,רעננה,דירה,4.5 חד׳,"140 מ""ר",קזן,10.0,מרכז דרום,3730000,6,קומה 3 מתוך 6,...,יש מחסן,משופץ,יש מיזוג אויר,יש מרפסת,יש ממ״ד,נגיש לנכים,2024-02-01 00:00:00,לא צויין,NaN,"לא למתווכים!, ללא תיווך!, מתווכים לא להתקשר! ד..."
6,ירושלים,דירה,2.5,52,גואטמאלה,NaN,קרית היובל,1870000,7,קומה 1 מתוך 9,...,False,משופץ,True,False,True,False,לא צויין,לא צויין,NaN,הבית בן כ-6 שנים. חניה פרטית. בית מושקע עם מטב...
7,תל אביב,דירה,NaN,"30 מ""ר",דיזנגוף,342.0,הצפון הישן החלק הצפוני,2290000,5,קומה 4 מתוך 4,...,אין מחסן,שמור,יש מיזוג אויר,אין מרפסת,אין ממ״ד,False,מיידי,מלא,12.0,"נוף מהמם לים!! בנין שמור וצבוע, אינטרקום וקוד ..."
8,יהוד מונוסון,דירה,4.5 חד׳,"100 מ""ר",שדרות מנחם,NaN,קרית סביונים,3150000,12,קומה 10 מתוך 12,...,False,ישן,True,True,False,False,2023-08-01 00:00:00,לא צויין,NaN,למכירה בשיווק בלעדי | דירת ארבעה וחצי חדרים. ב...
9,נוף הגליל,דירה,5,120,אריאל שרון,20.0,בן גוריון,1430000,3,קומה 3 מתוך 4,...,False,משופץ,True,False,False,False,גמיש,לא צויין,NaN,"דירת 5 חדרים משופצת. מרווחת, מוארת ומאווררת, מ..."


### Get basic information about the data.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               11 non-null     object 
 1   type               11 non-null     object 
 2   room_number        10 non-null     object 
 3   Area               11 non-null     object 
 4   Street             11 non-null     object 
 5   number_in_street   8 non-null      float64
 6   city_area          11 non-null     object 
 7   price              11 non-null     object 
 8   num_of_images      11 non-null     int64  
 9   floor_out_of       11 non-null     object 
 10  hasElevator        11 non-null     object 
 11  hasParking         11 non-null     object 
 12  hasBars            11 non-null     object 
 13  hasStorage         11 non-null     object 
 14  condition          11 non-null     object 
 15  hasAirCondition    11 non-null     object 
 16  hasBalcony         11 non-nu

In [4]:
data.isnull().sum().sort_values()

City                 0
furniture            0
entranceDate         0
handicapFriendly     0
hasMamad             0
hasBalcony           0
hasAirCondition      0
condition            0
hasStorage           0
hasBars              0
hasParking           0
floor_out_of         0
num_of_images        0
price                0
city_area            0
Street               0
Area                 0
type                 0
hasElevator          0
description          0
room_number          1
number_in_street     3
publishedDays        9
dtype: int64

In [5]:
data.columns

Index(['City', 'type', 'room_number', 'Area', 'Street', 'number_in_street',
       'city_area', 'price', 'num_of_images', 'floor_out_of', 'hasElevator ',
       'hasParking ', 'hasBars ', 'hasStorage ', 'condition ',
       'hasAirCondition ', 'hasBalcony ', 'hasMamad ', 'handicapFriendly ',
       'entranceDate ', 'furniture ', 'publishedDays ', 'description '],
      dtype='object')

# stage 2 - Clean and prepare the data.

In [6]:
data['publishedDays '] = data['publishedDays '].astype(str).str.extract(r'(\d+\.?\d*)')

In [7]:
if 'Unnamed: 23' in data.columns:
    data.drop('Unnamed: 23', axis=1, inplace=True)
data.isnull().sum().sort_values()

City                 0
furniture            0
entranceDate         0
handicapFriendly     0
hasMamad             0
hasBalcony           0
hasAirCondition      0
condition            0
hasStorage           0
hasBars              0
hasParking           0
floor_out_of         0
num_of_images        0
price                0
city_area            0
Street               0
Area                 0
type                 0
hasElevator          0
description          0
room_number          1
number_in_street     3
publishedDays        9
dtype: int64

In [8]:
data['City'] = data['City'].str.replace('נהרייה', 'נהריה')
for col in ['City']:
    if data[col].dtype == 'object':
        data[col] = data[col].str.strip()

In [9]:
data = data.astype({'room_number':str})
data['room_number'] = data['room_number'].str.extract(r'([-+]?\d*\.?\d+)')

In [10]:
data = data.dropna(subset=['price']) 

In [11]:
def fix_column(df, column_name):
    # Create a copy of the column
    df = df.astype({column_name:str})
    column_copy = df[column_name].copy()
    column_copy = column_copy.str.extract(r"(\d+)", expand=False)
    column_copy = column_copy.astype('Int64')
    df.loc[:, column_name] = column_copy.copy()

    return df

data = fix_column(data, 'price')
data = fix_column(data, 'Area')

# Print the modified DataFrame
data.head()

<ipython-input-11-44c90a619fe1>:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, column_name] = column_copy.copy()
<ipython-input-11-44c90a619fe1>:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, column_name] = column_copy.copy()


,City,type,room_number,Area,Street,number_in_street,city_area,price,num_of_images,floor_out_of,...,hasStorage,condition,hasAirCondition,hasBalcony,hasMamad,handicapFriendly,entranceDate,furniture,publishedDays,description
0,פתח תקווה,בית פרטי,4,600,עזרא ונחמיה,44.0,עמישב,6999999,2,קומת קרקע,...,False,משופץ,True,False,False,False,לא צויין,מלא,NaN,"דירת מרווחת ומוארת במצב מצוין, 4 חדרים גדולים,..."
1,דימונה,דירה,2,45,דיין משה,117.0,נאות קטיף,520000,0,קומה 1 מתוך 4,...,False,לא צויין,True,False,False,False,לא צויין,מלא,11.0,אזור טוב ושקט בדימונה.שכנים נוחים.הדירה מושכרת...
2,ירושלים,דירה,3,94,רבדים,6.0,ארנונה (תלפיות),3450000,11,קומה 8 מתוך 8,...,כן,משופץ,כן,כן,לא,כן,לא צויין,חלקי,NaN,"הדירה משופצת ומושקעת , נמצאת בקרבת תחבורה ציב..."
3,הרצליה,דירה,5,135,פינס,5.0,הרצליה הצעירה,3800000,0,קומה 1 מתוך 3,...,יש מחסן,משופץ,יש מיזוג אויר,אין מרפסת,אין ממ״ד,לא נגיש לנכים,לא צויין,לא צויין,NaN,דירה גדולה מאוד יחידה בקומה. משופצת כמו חדשה. ...
4,רעננה,דירת גן,5,139,עלי מוהר,9.0,נווה זמר,5555555,4,קומת קרקע,...,יש מחסן,חדש מקבלן,יש מיזוג אויר,אין מרפסת,יש ממ״ד,לא נגיש לנכים,2025-02-01 00:00:00,ללא,NaN,דירת גן בפרוייקט יוקרתי בבניין בוטיק רק 12 דיר...


In [12]:
# Function that deletes punctuation inside the information columns - except for the dot of float numbers.
def remove_punctuation(data_name, columns_list):
    cleaned_data = data_name.copy()

    # Define a translation table to replace punctuation with space
    translation_table = str.maketrans('', '', string.punctuation.replace('.', '') + '\n')

    # Iterate over the specified columns
    for column in columns_list:
        # Remove punctuation from the column values and replace with space
        cleaned_data[column] = cleaned_data[column].astype(str).apply(lambda x: re.sub(r'(?<!\d|\.)[{0}]'.format(string.punctuation.replace('"', '')), ' ', x).translate(translation_table))
        
        # Replace double quotation marks with an empty string
        cleaned_data[column] = cleaned_data[column].astype(str).apply(lambda x: x.replace('"', ''))

    return cleaned_data

data = remove_punctuation(data, ['Street', 'city_area','description '])
data


,City,type,room_number,Area,Street,number_in_street,city_area,price,num_of_images,floor_out_of,...,hasStorage,condition,hasAirCondition,hasBalcony,hasMamad,handicapFriendly,entranceDate,furniture,publishedDays,description
0,פתח תקווה,בית פרטי,4,600,עזרא ונחמיה,44.0,עמישב,6999999,2,קומת קרקע,...,False,משופץ,True,False,False,False,לא צויין,מלא,NaN,דירת מרווחת ומוארת במצב מצוין 4 חדרים גדולים ...
1,דימונה,דירה,2,45,דיין משה,117.0,נאות קטיף,520000,0,קומה 1 מתוך 4,...,False,לא צויין,True,False,False,False,לא צויין,מלא,11.0,אזור טוב ושקט בדימונה שכנים נוחים הדירה מושכרת...
2,ירושלים,דירה,3,94,רבדים,6.0,ארנונה תלפיות,3450000,11,קומה 8 מתוך 8,...,כן,משופץ,כן,כן,לא,כן,לא צויין,חלקי,NaN,הדירה משופצת ומושקעת נמצאת בקרבת תחבורה ציב...
3,הרצליה,דירה,5,135,פינס,5.0,הרצליה הצעירה,3800000,0,קומה 1 מתוך 3,...,יש מחסן,משופץ,יש מיזוג אויר,אין מרפסת,אין ממ״ד,לא נגיש לנכים,לא צויין,לא צויין,NaN,דירה גדולה מאוד יחידה בקומה משופצת כמו חדשה ...
4,רעננה,דירת גן,5,139,עלי מוהר,9.0,נווה זמר,5555555,4,קומת קרקע,...,יש מחסן,חדש מקבלן,יש מיזוג אויר,אין מרפסת,יש ממ״ד,לא נגיש לנכים,2025-02-01 00:00:00,ללא,NaN,דירת גן בפרוייקט יוקרתי בבניין בוטיק רק 12 דיר...
5,רעננה,דירה,4.5,140,קזן,10.0,מרכז דרום,3730000,6,קומה 3 מתוך 6,...,יש מחסן,משופץ,יש מיזוג אויר,יש מרפסת,יש ממ״ד,נגיש לנכים,2024-02-01 00:00:00,לא צויין,NaN,לא למתווכים ללא תיווך מתווכים לא להתקשר ד...
6,ירושלים,דירה,2.5,52,גואטמאלה,NaN,קרית היובל,1870000,7,קומה 1 מתוך 9,...,False,משופץ,True,False,True,False,לא צויין,לא צויין,NaN,הבית בן כ 6 שנים חניה פרטית בית מושקע עם מטב...
7,תל אביב,דירה,NaN,30,דיזנגוף,342.0,הצפון הישן החלק הצפוני,2290000,5,קומה 4 מתוך 4,...,אין מחסן,שמור,יש מיזוג אויר,אין מרפסת,אין ממ״ד,False,מיידי,מלא,12.0,נוף מהמם לים בנין שמור וצבוע אינטרקום וקוד ...
8,יהוד מונוסון,דירה,4.5,100,שדרות מנחם,NaN,קרית סביונים,3150000,12,קומה 10 מתוך 12,...,False,ישן,True,True,False,False,2023-08-01 00:00:00,לא צויין,NaN,למכירה בשיווק בלעדי דירת ארבעה וחצי חדרים ב...
9,נוף הגליל,דירה,5,120,אריאל שרון,20.0,בן גוריון,1430000,3,קומה 3 מתוך 4,...,False,משופץ,True,False,False,False,גמיש,לא צויין,NaN,דירת 5 חדרים משופצת מרווחת מוארת ומאווררת מ...


### Function to extract floor number and total floors

In [13]:
def extract_floor_info(value):
    value = str(value)  # Convert to string
    if 'קרקע' in value:
        return 0,1  # Ground floor with no total floors
    elif 'קומה' in value:
        match = re.search(r'(\d+)\s*מתוך\s*(\d+)', value)
        if match:
            current_floor = int(match.group(1))
            total_floors = int(match.group(2))
            return current_floor, total_floors
    return None, None

# Add floor and total_floors columns
data['floor'], data['total_floors'] = zip(*data['floor_out_of'].map(extract_floor_info))

# Convert the columnד data to numeric type
data['floor'] = data['floor'].astype(pd.Int64Dtype())
data['total_floors'] = data['total_floors'].astype(pd.Int64Dtype())

# Replace <NA> values with None.
data['floor'] = data['floor'].astype(object).where(data['floor'].notnull(), None)
data['total_floors'] = data['total_floors'].astype(object).where(data['total_floors'].notnull(), None)

# Print the modified DataFrame
data


,City,type,room_number,Area,Street,number_in_street,city_area,price,num_of_images,floor_out_of,...,hasAirCondition,hasBalcony,hasMamad,handicapFriendly,entranceDate,furniture,publishedDays,description,floor,total_floors
0,פתח תקווה,בית פרטי,4,600,עזרא ונחמיה,44.0,עמישב,6999999,2,קומת קרקע,...,True,False,False,False,לא צויין,מלא,NaN,דירת מרווחת ומוארת במצב מצוין 4 חדרים גדולים ...,0,1
1,דימונה,דירה,2,45,דיין משה,117.0,נאות קטיף,520000,0,קומה 1 מתוך 4,...,True,False,False,False,לא צויין,מלא,11.0,אזור טוב ושקט בדימונה שכנים נוחים הדירה מושכרת...,1,4
2,ירושלים,דירה,3,94,רבדים,6.0,ארנונה תלפיות,3450000,11,קומה 8 מתוך 8,...,כן,כן,לא,כן,לא צויין,חלקי,NaN,הדירה משופצת ומושקעת נמצאת בקרבת תחבורה ציב...,8,8
3,הרצליה,דירה,5,135,פינס,5.0,הרצליה הצעירה,3800000,0,קומה 1 מתוך 3,...,יש מיזוג אויר,אין מרפסת,אין ממ״ד,לא נגיש לנכים,לא צויין,לא צויין,NaN,דירה גדולה מאוד יחידה בקומה משופצת כמו חדשה ...,1,3
4,רעננה,דירת גן,5,139,עלי מוהר,9.0,נווה זמר,5555555,4,קומת קרקע,...,יש מיזוג אויר,אין מרפסת,יש ממ״ד,לא נגיש לנכים,2025-02-01 00:00:00,ללא,NaN,דירת גן בפרוייקט יוקרתי בבניין בוטיק רק 12 דיר...,0,1
5,רעננה,דירה,4.5,140,קזן,10.0,מרכז דרום,3730000,6,קומה 3 מתוך 6,...,יש מיזוג אויר,יש מרפסת,יש ממ״ד,נגיש לנכים,2024-02-01 00:00:00,לא צויין,NaN,לא למתווכים ללא תיווך מתווכים לא להתקשר ד...,3,6
6,ירושלים,דירה,2.5,52,גואטמאלה,NaN,קרית היובל,1870000,7,קומה 1 מתוך 9,...,True,False,True,False,לא צויין,לא צויין,NaN,הבית בן כ 6 שנים חניה פרטית בית מושקע עם מטב...,1,9
7,תל אביב,דירה,NaN,30,דיזנגוף,342.0,הצפון הישן החלק הצפוני,2290000,5,קומה 4 מתוך 4,...,יש מיזוג אויר,אין מרפסת,אין ממ״ד,False,מיידי,מלא,12.0,נוף מהמם לים בנין שמור וצבוע אינטרקום וקוד ...,4,4
8,יהוד מונוסון,דירה,4.5,100,שדרות מנחם,NaN,קרית סביונים,3150000,12,קומה 10 מתוך 12,...,True,True,False,False,2023-08-01 00:00:00,לא צויין,NaN,למכירה בשיווק בלעדי דירת ארבעה וחצי חדרים ב...,10,12
9,נוף הגליל,דירה,5,120,אריאל שרון,20.0,בן גוריון,1430000,3,קומה 3 מתוך 4,...,True,False,False,False,גמיש,לא צויין,NaN,דירת 5 חדרים משופצת מרווחת מוארת ומאווררת מ...,3,4


In [14]:
data = data.drop(['floor_out_of'], axis=1)

### We will check the values in the column "entranceDate".

In [15]:
data['entranceDate '].unique()

array(['לא צויין', datetime.datetime(2025, 2, 1, 0, 0),
       datetime.datetime(2024, 2, 1, 0, 0), 'מיידי',
       datetime.datetime(2023, 8, 1, 0, 0), 'גמיש'], dtype=object)

In [16]:
import datetime

data['entranceDate '] = data['entranceDate '].replace('גמיש', 'flexible')
data['entranceDate '] = data['entranceDate '].replace('גמיש ', 'flexible')
data['entranceDate '] = data['entranceDate '].replace('לא צויין', 'not_defined')
data['entranceDate '] = data['entranceDate '].replace('מיידי', 'Less_than_6 months')

valid_dates_mask = pd.to_datetime(data['entranceDate '], format='%d/%m/%Y', errors='coerce').notna()
valid_dates_mask |= pd.to_datetime(data['entranceDate '], format='%Y-%m-%d %H:%M:%S', errors='coerce').notna()

current_date = datetime.date.today()

# Create a temporary column for time difference
data['time_difference'] = pd.NaT

# Calculate time difference only for valid dates
valid_dates = pd.to_datetime(data.loc[valid_dates_mask, 'entranceDate '], errors='coerce')
data.loc[valid_dates_mask, 'time_difference'] = (valid_dates - pd.to_datetime(current_date)).dt.days / 30

# Define bins and labels
bins = [-float('inf'), 6, 12, float('inf')]
labels = ['Less_than_6 months', 'months_6_12', 'Above_year']

# Categorize valid dates based on time difference
data.loc[valid_dates_mask, 'entranceDate '] = pd.cut(data.loc[valid_dates_mask, 'time_difference'], bins=bins, labels=labels)

# Replace remaining values with appropriate labels
data['entranceDate '] = data['entranceDate '].fillna('invalid_value')

# Drop the unnecessary column
data = data.drop(['time_difference'], axis=1)

### Print unique values in the 'years_difference' column

In [17]:
data['entranceDate '].unique()

array(['not_defined', 'Above_year', 'months_6_12', 'Less_than_6 months',
       'flexible'], dtype=object)

### Convert the TRUE\ FALSE columns to 0\ 1

In [18]:
def convert_boolean_to_binary(dataframe, columns, positive_words, negative_words):
    for column in columns:
        for word in positive_words:
            dataframe[column] = np.where(dataframe[column].str.contains(word, case=False, na=False), 1, dataframe[column])
        for word in negative_words:
            dataframe[column] = np.where(dataframe[column].str.contains(word, case=False, na=False), 0, dataframe[column])
        dataframe[column] = pd.to_numeric(dataframe[column], errors='coerce').fillna(0).astype(int)
    return dataframe


positive_words = ['כן', 'יש', 'נגיש', 'TRUE']
negative_words = ['לא', 'אין', 'FALSE']
boolean_columns = ['hasElevator ', 'hasParking ', 'hasBars ', 'hasStorage ', 'hasAirCondition ', 'hasBalcony ', 'hasMamad ', 'handicapFriendly ']

data = convert_boolean_to_binary(data, boolean_columns ,positive_words, negative_words)
data.tail()

,City,type,room_number,Area,Street,number_in_street,city_area,price,num_of_images,hasElevator,...,hasAirCondition,hasBalcony,hasMamad,handicapFriendly,entranceDate,furniture,publishedDays,description,floor,total_floors
6,ירושלים,דירה,2.5,52,גואטמאלה,NaN,קרית היובל,1870000,7,1,...,1,0,1,0,not_defined,לא צויין,NaN,הבית בן כ 6 שנים חניה פרטית בית מושקע עם מטב...,1,9
7,תל אביב,דירה,NaN,30,דיזנגוף,342.0,הצפון הישן החלק הצפוני,2290000,5,0,...,1,0,0,0,Less_than_6 months,מלא,12.0,נוף מהמם לים בנין שמור וצבוע אינטרקום וקוד ...,4,4
8,יהוד מונוסון,דירה,4.5,100,שדרות מנחם,NaN,קרית סביונים,3150000,12,1,...,1,1,0,0,Less_than_6 months,לא צויין,NaN,למכירה בשיווק בלעדי דירת ארבעה וחצי חדרים ב...,10,12
9,נוף הגליל,דירה,5,120,אריאל שרון,20.0,בן גוריון,1430000,3,0,...,1,0,0,0,flexible,לא צויין,NaN,דירת 5 חדרים משופצת מרווחת מוארת ומאווררת מ...,3,4
10,ירושלים,דירת גן,4,100,רמון יוסף צבי,NaN,רמת בית הכרם,4,3,0,...,1,0,1,0,not_defined,ללא,NaN,הדירה משופצת ומושקעת נמצאת בקרבת תחבורה ציב...,1,1


In [19]:
data = data.dropna(subset=['price'])

In [20]:
for col in ['City', 'type','Street' ,'condition ']:
    if data[col].dtype == 'object':
        data[col] = data[col].str.strip()

In [21]:
data.dtypes

City                  object
type                  object
room_number           object
Area                   Int64
Street                object
number_in_street     float64
city_area             object
price                  Int64
num_of_images          int64
hasElevator            int32
hasParking             int32
hasBars                int32
hasStorage             int32
condition             object
hasAirCondition        int32
hasBalcony             int32
hasMamad               int32
handicapFriendly       int32
entranceDate          object
furniture             object
publishedDays         object
description           object
floor                 object
total_floors          object
dtype: object

In [22]:
data['Area'] = pd.to_numeric(data['Area'], errors='coerce')
data['room_number'] = pd.to_numeric(data['room_number'], errors='coerce')
data['number_in_street'] = pd.to_numeric(data['number_in_street'], errors='coerce')
data['publishedDays '] = pd.to_numeric(data['publishedDays '], errors='coerce')
col = ['floor','total_floors','price']
data[col] = data[col].astype(float)

In [23]:
data.dtypes

City                  object
type                  object
room_number          float64
Area                   Int64
Street                object
number_in_street     float64
city_area             object
price                float64
num_of_images          int64
hasElevator            int32
hasParking             int32
hasBars                int32
hasStorage             int32
condition             object
hasAirCondition        int32
hasBalcony             int32
hasMamad               int32
handicapFriendly       int32
entranceDate          object
furniture             object
publishedDays        float64
description           object
floor                float64
total_floors         float64
dtype: object

In [24]:
data

,City,type,room_number,Area,Street,number_in_street,city_area,price,num_of_images,hasElevator,...,hasAirCondition,hasBalcony,hasMamad,handicapFriendly,entranceDate,furniture,publishedDays,description,floor,total_floors
0,פתח תקווה,בית פרטי,4.0,600,עזרא ונחמיה,44.0,עמישב,6999999.0,2,0,...,1,0,0,0,not_defined,מלא,NaN,דירת מרווחת ומוארת במצב מצוין 4 חדרים גדולים ...,0.0,1.0
1,דימונה,דירה,2.0,45,דיין משה,117.0,נאות קטיף,520000.0,0,0,...,1,0,0,0,not_defined,מלא,11.0,אזור טוב ושקט בדימונה שכנים נוחים הדירה מושכרת...,1.0,4.0
2,ירושלים,דירה,3.0,94,רבדים,6.0,ארנונה תלפיות,3450000.0,11,1,...,1,1,0,1,not_defined,חלקי,NaN,הדירה משופצת ומושקעת נמצאת בקרבת תחבורה ציב...,8.0,8.0
3,הרצליה,דירה,5.0,135,פינס,5.0,הרצליה הצעירה,3800000.0,0,0,...,1,0,0,1,not_defined,לא צויין,NaN,דירה גדולה מאוד יחידה בקומה משופצת כמו חדשה ...,1.0,3.0
4,רעננה,דירת גן,5.0,139,עלי מוהר,9.0,נווה זמר,5555555.0,4,1,...,1,0,1,1,Above_year,ללא,NaN,דירת גן בפרוייקט יוקרתי בבניין בוטיק רק 12 דיר...,0.0,1.0
5,רעננה,דירה,4.5,140,קזן,10.0,מרכז דרום,3730000.0,6,1,...,1,1,1,1,months_6_12,לא צויין,NaN,לא למתווכים ללא תיווך מתווכים לא להתקשר ד...,3.0,6.0
6,ירושלים,דירה,2.5,52,גואטמאלה,NaN,קרית היובל,1870000.0,7,1,...,1,0,1,0,not_defined,לא צויין,NaN,הבית בן כ 6 שנים חניה פרטית בית מושקע עם מטב...,1.0,9.0
7,תל אביב,דירה,NaN,30,דיזנגוף,342.0,הצפון הישן החלק הצפוני,2290000.0,5,0,...,1,0,0,0,Less_than_6 months,מלא,12.0,נוף מהמם לים בנין שמור וצבוע אינטרקום וקוד ...,4.0,4.0
8,יהוד מונוסון,דירה,4.5,100,שדרות מנחם,NaN,קרית סביונים,3150000.0,12,1,...,1,1,0,0,Less_than_6 months,לא צויין,NaN,למכירה בשיווק בלעדי דירת ארבעה וחצי חדרים ב...,10.0,12.0
9,נוף הגליל,דירה,5.0,120,אריאל שרון,20.0,בן גוריון,1430000.0,3,0,...,1,0,0,0,flexible,לא צויין,NaN,דירת 5 חדרים משופצת מרווחת מוארת ומאווררת מ...,3.0,4.0


### checked outliers and chose to refer to a certain price range according to certain cities.

In [25]:
data = data.dropna(subset = ['condition '])
data = data[data['price'] <= 15000000]
data = data.loc[(data['City'] != 'גבעת שמואל') | (data['price'] >= 1000000)]
data = data.loc[(data['City'] != 'זכרון יעקב') | (data['price'] >= 1000000)]
data = data.loc[(data['City'] != 'כפר סבא') | (data['price'] >= 1000000)]
data = data.loc[(data['City'] != 'הרצליה') | (data['price'] >= 1000000)]
data = data.loc[(data['City'] != 'דימונה') | (data['price'] < 6000000)]
data = data.loc[(data['City'] != 'ראשון לציון') | (data['price'] < 5000000)]
data.isnull().sum().sort_values()

City                 0
description          0
furniture            0
entranceDate         0
handicapFriendly     0
hasMamad             0
hasBalcony           0
hasAirCondition      0
condition            0
hasStorage           0
floor                0
hasBars              0
hasElevator          0
num_of_images        0
price                0
city_area            0
Street               0
Area                 0
type                 0
hasParking           0
total_floors         0
room_number          1
number_in_street     3
publishedDays        9
dtype: int64

# stage 3

### separate the variable we want to predict from the decision variables

In [26]:
x = data.drop("price", axis=1)
y = data.price.astype(float)

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [28]:
X_train.isnull().sum().sort_values()

City                 0
description          0
furniture            0
entranceDate         0
handicapFriendly     0
hasMamad             0
hasBalcony           0
hasAirCondition      0
condition            0
floor                0
hasStorage           0
hasParking           0
hasElevator          0
num_of_images        0
city_area            0
Street               0
Area                 0
type                 0
hasBars              0
total_floors         0
room_number          1
number_in_street     2
publishedDays        5
dtype: int64

In [29]:
X_train.dtypes

City                  object
type                  object
room_number          float64
Area                   Int64
Street                object
number_in_street     float64
city_area             object
num_of_images          int64
hasElevator            int32
hasParking             int32
hasBars                int32
hasStorage             int32
condition             object
hasAirCondition        int32
hasBalcony             int32
hasMamad               int32
handicapFriendly       int32
entranceDate          object
furniture             object
publishedDays        float64
description           object
floor                float64
total_floors         float64
dtype: object

In [30]:
null_col = X_train.columns[X_train.isnull().any()]
simp = SimpleImputer(strategy='most_frequent', add_indicator=False)
simp.fit(X_train[null_col])
X_train[null_col] = simp.transform(X_train[null_col])
X_train.isnull().sum().sort_values()

<ipython-input-30-cf157198e453>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[null_col] = simp.transform(X_train[null_col])


City                 0
description          0
publishedDays        0
furniture            0
entranceDate         0
handicapFriendly     0
hasMamad             0
hasBalcony           0
hasAirCondition      0
condition            0
floor                0
hasStorage           0
hasParking           0
hasElevator          0
num_of_images        0
city_area            0
number_in_street     0
Street               0
Area                 0
room_number          0
type                 0
hasBars              0
total_floors         0
dtype: int64

In [31]:
X_train.dtypes

City                  object
type                  object
room_number          float64
Area                   Int64
Street                object
number_in_street     float64
city_area             object
num_of_images          int64
hasElevator            int32
hasParking             int32
hasBars                int32
hasStorage             int32
condition             object
hasAirCondition        int32
hasBalcony             int32
hasMamad               int32
handicapFriendly       int32
entranceDate          object
furniture             object
publishedDays        float64
description           object
floor                float64
total_floors         float64
dtype: object

In [32]:
X_train

,City,type,room_number,Area,Street,number_in_street,city_area,num_of_images,hasElevator,hasParking,...,hasAirCondition,hasBalcony,hasMamad,handicapFriendly,entranceDate,furniture,publishedDays,description,floor,total_floors
2,ירושלים,דירה,3.0,94,רבדים,6.0,ארנונה תלפיות,11,1,1,...,1,1,0,1,not_defined,חלקי,11.0,הדירה משופצת ומושקעת נמצאת בקרבת תחבורה ציב...,8.0,8.0
1,דימונה,דירה,2.0,45,דיין משה,117.0,נאות קטיף,0,0,0,...,1,0,0,0,not_defined,מלא,11.0,אזור טוב ושקט בדימונה שכנים נוחים הדירה מושכרת...,1.0,4.0
8,יהוד מונוסון,דירה,4.5,100,שדרות מנחם,5.0,קרית סביונים,12,1,0,...,1,1,0,0,Less_than_6 months,לא צויין,11.0,למכירה בשיווק בלעדי דירת ארבעה וחצי חדרים ב...,10.0,12.0
4,רעננה,דירת גן,5.0,139,עלי מוהר,9.0,נווה זמר,4,1,1,...,1,0,1,1,Above_year,ללא,11.0,דירת גן בפרוייקט יוקרתי בבניין בוטיק רק 12 דיר...,0.0,1.0
7,תל אביב,דירה,5.0,30,דיזנגוף,342.0,הצפון הישן החלק הצפוני,5,0,0,...,1,0,0,0,Less_than_6 months,מלא,12.0,נוף מהמם לים בנין שמור וצבוע אינטרקום וקוד ...,4.0,4.0
3,הרצליה,דירה,5.0,135,פינס,5.0,הרצליה הצעירה,0,0,0,...,1,0,0,1,not_defined,לא צויין,11.0,דירה גדולה מאוד יחידה בקומה משופצת כמו חדשה ...,1.0,3.0
6,ירושלים,דירה,2.5,52,גואטמאלה,5.0,קרית היובל,7,1,0,...,1,0,1,0,not_defined,לא צויין,11.0,הבית בן כ 6 שנים חניה פרטית בית מושקע עם מטב...,1.0,9.0


In [33]:
numerical_features = ['Area',
 'number_in_street',
 'num_of_images',
 'hasElevator ',
 'hasParking ',
 'hasBars ',
 'hasStorage ',
 'hasAirCondition ',
 'hasBalcony ',
 'hasMamad ',
 'handicapFriendly ',
 'publishedDays ',
 'floor',
 'total_floors',
 'room_number']

# Calculate VIF for each numeric column
vif_data = pd.DataFrame()
vif_data["Variable"] = numerical_features
vif_data["VIF"] = [variance_inflation_factor(X_train[numerical_features].values.astype(np.float64), i)
                   for i in range(len(numerical_features))]

vif_data


C:\Users\amitf\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\amitf\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\amitf\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\amitf\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\amitf\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\amitf\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: Runtime

,Variable,VIF
0,Area,inf
1,number_in_street,inf
2,num_of_images,inf
3,hasElevator,inf
4,hasParking,inf
5,hasBars,inf
6,hasStorage,inf
7,hasAirCondition,0.0
8,hasBalcony,inf
9,hasMamad,inf


### Based on VIF you can see that there are columns that can be removed from the training set

In [34]:
filtered_variables = vif_data.loc[vif_data['VIF'] >= 5, 'Variable'].values
X_train = X_train.drop(filtered_variables, axis=1)
X_train.columns

Index(['City', 'type', 'Street', 'city_area', 'condition ', 'hasAirCondition ',
       'entranceDate ', 'furniture ', 'description '],
      dtype='object')

### separate categorical columns from numeric columns to create a different pipeline for each type

In [35]:
num_cols = [col for col in X_train.columns if X_train[col].dtypes!='O' and (X_train[col].dtypes!='object')]
cat_cols = [col for col in X_train.columns if (X_train[col].dtypes=='O' or (X_train[col].dtypes=='object'))]

In [36]:
num_cols

['hasAirCondition ']

In [37]:
cat_cols

['City',
 'type',
 'Street',
 'city_area',
 'condition ',
 'entranceDate ',
 'furniture ',
 'description ']

In [38]:
numerical_pipeline = Pipeline([('numerical_imputation', SimpleImputer(strategy='most_frequent', add_indicator=False)),('scaling', StandardScaler())])

categorical_pipeline = Pipeline([('categorical_imputation', SimpleImputer(strategy='most_frequent', add_indicator=False, fill_value='missing')),('one_hot_encoding', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

column_transformer = ColumnTransformer([('numerical_preprocessing', numerical_pipeline, num_cols),('categorical_preprocessing', categorical_pipeline, cat_cols)], remainder='drop')

In [39]:
elastic_net_pipeline = Pipeline([('preprocessing', column_transformer),('elastic_net', ElasticNet(alpha=0.8, l1_ratio=1, random_state=42))])

In [40]:
cross_val = KFold(n_splits=10)

In [41]:
scores = cross_val_score(elastic_net_pipeline, x, y, cv=cross_val , scoring='neg_mean_squared_error')

In [42]:
scores = cross_val_score(elastic_net_pipeline, x, y, cv=cross_val , scoring='neg_mean_squared_error')
print("Cross-validation scores:", scores)
print("Average MSE:", np.mean(-scores))
rmse_scores = np.sqrt(-scores)
print("Average RMSE:", np.mean(rmse_scores))

Cross-validation scores: [-1.34032065e+13 -9.29954461e+12 -2.35965856e+10 -1.09373617e+13
 -4.95160006e+12 -7.45203094e+10 -2.03870247e+12 -2.75210211e+11
 -5.44621078e+12 -1.09903767e+13]
Average MSE: 5744032993689.876
Average RMSE: 2027085.9391895807
